In [1]:
# loaded_model = pickle.load("squeezenetfinetuned.pkl")
import sklearn 
import numpy as np
import joblib
import pickle
import torch

#Basma
# classifer = joblib.load("C:/Users/basma/Fourth Year/Graduation_Project/squeezenetfinetuned.pkl")

#Bassem
# classifer = joblib.load("squeezenetuntrained.pkl")

model = pickle.load(open('new squeezenet(finetuned).pkl', 'rb'))
model.eval()

c:\Users\Ahmed B\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [2]:
import cv2
import numpy as np

deleteCounterImage = 0

# Set threshold values for binary image and blob analysis
binary_threshold = 50
blob_min_area = 100

# Load the input image
image = cv2.imread("experiments/bad_evil_snake.png")

# Convert image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Otsu's thresholding method to get a binary image
_, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Invert the binary image
binary = cv2.bitwise_not(threshold)

# Perform blob analysis
contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours by x-coordinate and then by y-coordinate
# contours = sorted(contours, key=lambda c: (cv2.boundingRect(c)[0], cv2.boundingRect(c)[1]))
# Sort contours from top to bottom and left to right
contours = sorted(contours, key=lambda c: (cv2.boundingRect(c)[1], cv2.boundingRect(c)[0]))

# Iterate over the sorted contours
for i, contour in enumerate(contours):
    
    # Check if the area of the contour is large enough to be considered an object
    if cv2.contourArea(contour) > blob_min_area:
        # Get bounding box coordinates
        x, y, w, h = cv2.boundingRect(contour)

        # Crop the image using the bounding box coordinates
        crop_img = image[y:y+h, x:x+w]
        # Save the cropped image to a file
        filename = f"crop_{i}.png"
        cv2.imwrite(filename, crop_img)
        

In [3]:
from PIL import Image, ImageFilter
import glob

image_array = []
image_files = glob.glob("*crop*.png")
for i, image_file in enumerate(image_files):
    
    print("ana: ",i)
    # Load the input image
    image1 = cv2.imread(image_file)

    # Get the original dimensions of the image
    height, width, _ = image1.shape

    # Calculate the aspect ratio of the image
    aspect_ratio = width / height

    # Set the desired output size (height or width)
    output_size = 224

    # Calculate the other dimension based on the aspect ratio
    if width > height:
        new_width = output_size
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = output_size
        new_width = int(new_height * aspect_ratio)

    # Resize the image
    resized_image = cv2.resize(image1, (new_width, new_height))

    # Calculate the coordinates to copy the resized image onto the new image
    x_offset = int((output_size - new_width) / 2)
    y_offset = int((output_size - new_height) / 2)

    # Save the resized image to a file
    cv2.imwrite(f"resize{i}.png", resized_image)

    # Load the resized image from file
    resized_image = cv2.imread(f"resize{i}.png")

    # Convert image to grayscale
    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    # Apply Otsu's thresholding method to get a binary image
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
    # Invert the binary image
    inv = cv2.bitwise_not(thresh)
    # Save the segmented image to a file
    cv2.imwrite(f"seg_img{i}.png", inv)
    # Load the segmented image from file
    seg_img = cv2.imread(f"seg_img{i}.png")

    # Create a new blank image with the desired output size
    new_image = np.zeros((output_size, output_size, 3), np.uint8)
    new_image[:, :] = (0, 0, 0)  # fill with black pixels

    # Copy the segmented image onto the new image with black pixels
    new_image[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = seg_img

    # Save the new image to a file
    cv2.imwrite(f"final{i}.png", new_image)
    deleteCounterImage = deleteCounterImage+1
    image_array.append(new_image)


ana:  0
ana:  1
ana:  2
ana:  3
ana:  4
ana:  5
ana:  6
ana:  7
ana:  8


In [4]:
import torchvision.transforms as transforms
from PIL import Image
from PIL import ImageFile
import cv2

results = ""
#Basma
for i in range(deleteCounterImage):
    print(deleteCounterImage)
    input_image = Image.open(f'final{i}.png').convert('RGB')
    # input_image = Image.open('C:/Users/basma/Fourth Year/Graduation_Project/SegmentedDatasetAgain/train/A040/A040_1.png').convert('RGB')
    #Bassem
    # input_image = Image.open('D:/Ahmed Bassem/MIU/Year 4/Graduation Project/Datasets/SegmentedDataset/SegmentedDataset/test/O001/O001_1.png').convert('RGB')

    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        # transforms.Pad(50),
    ])
    input_tensor = preprocess(input_image)
    # input_tensor = preprocess(Image.fromarray(i))
    input_batch = input_tensor.unsqueeze(0)
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)

    # Tensor of shape 1000, with confidence scores over Aegyptos' 1072 classes
    print(output[0])
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Read the categories
    #Basma
    with open("classes_names.txt", "r") as f:

    #Bassem
    # with open("D:/Ahmed Bassem/MIU\Year 4/Graduation Project/classes_names.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    # Show top categories per image
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        print(categories[top5_catid[i]], top5_prob[i].item())

    top1 = categories[top5_catid[0]]
    print(top1)

    class_name = 'T035'
    class_index = categories.index(class_name)
    class_prob = probabilities[class_index] * 100
    print(f"Percentage of confidence for {class_name}: {class_prob:.2f}%")

    # results = results + top1 + " "
    results += " "+ top1
updated_results = results.replace(" ", "", 1)
    # results.append(top1)
print(updated_results)
choose_dictionary_list=deleteCounterImage-1
print(choose_dictionary_list)

9
tensor([14.3750, 16.7328, 16.2243,  ...,  7.7657,  6.8522,  9.4124])
I009 0.35691094398498535
D035 0.24751420319080353
D030 0.08463063836097717
D044 0.0448552705347538
V011B 0.038954958319664
I009
Percentage of confidence for T035: 0.00%
9


c:\Users\Ahmed B\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


tensor([ 9.8701, 16.6496, 17.9634,  ..., 15.7820, 11.0325, 10.3741])
N035A 0.6072943806648254
N035 0.38613319396972656
F032 0.006270901765674353
NU011 9.653968299971893e-05
E002 6.764876161469147e-05
N035A
Percentage of confidence for T035: 0.00%
9
tensor([11.4104,  6.3771,  3.9526,  ...,  1.9068,  1.7442,  4.4242])
U036 0.621677577495575
T035 0.3487195372581482
S002 0.013632675632834435
AA007B 0.002668979112058878
F009 0.0019910582341253757
U036
Percentage of confidence for T035: 34.87%
9
tensor([4.9667, 2.5005, 4.1679,  ..., 3.9314, 4.9319, 7.1141])
U028 0.12581658363342285
AA007B 0.10297230631113052
S043 0.09682084619998932
X007 0.09406831115484238
W004 0.03190876543521881
U028
Percentage of confidence for T035: 0.07%
9
tensor([17.5514, 25.4380, 18.7231,  ..., 11.3540,  7.8485, 11.1756])
I010 0.8909083604812622
I011 0.06865179538726807
V012B 0.027964770793914795
V011C 0.003634243505075574
F020 0.0033053955994546413
I010
Percentage of confidence for T035: 0.00%
9
tensor([12.3210, 23.

In [8]:
# for i in range(deleteCounterImage):
#     print(i)
#     if(updated_results[i] == 'A'):
#         print("A")
#         print(i)
        

0
1
2


In [5]:
# import matching_algorithm as ma
# print(ma.dictionary_matching(updated_results))

##NEW CODE
import matching_algorithm3 as ma
print(ma.dictionary_matching(updated_results,choose_dictionary_list))


Snake


In [6]:
import os

# Delete previously created images
cropImageRemovals = glob.glob("*crop*.png")
for i in range(deleteCounterImage):
    removeCrop = cropImageRemovals
    removeSegImg = f"seg_img{i}.png"
    removeFinal = f"final{i}.png"
    removeResize = f"resize{i}.png"
    try:
        os.remove(removeCrop[i])
        os.remove(removeSegImg)
        os.remove(removeFinal)
        os.remove(removeResize)
    except FileNotFoundError:
        pass

In [6]:
import matching_algorithm_pronunciation as ma
print(ma.dictionary_matching_pronunciation(updated_results))

لا يوجد صوت


In [7]:
import googletrans
from googletrans import *

In [8]:
match = ma.dictionary_matching(updated_results)

AttributeError: module 'matching_algorithm_pronunciation' has no attribute 'dictionary_matching'

In [9]:
translator = googletrans.Translator()
translate=translator.translate(match,dest='arabic')
print(translate.text)

NameError: name 'match' is not defined